In [54]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize


import matplotlib.pyplot as plt
import seaborn as sns

In [55]:
# 设置中文字体为宋体
plt.rcParams['font.sans-serif'] = ['SimHei']
# 设置英文字体为新罗马
plt.rcParams['font.serif'] = ['Times New Roman']
# 字体大小
plt.rcParams['font.size'] = 12  
# 正常显示负号
plt.rcParams['axes.unicode_minus'] = False 

In [56]:
df1 = pd.read_csv('中间数据集\\按品类的日粒度时序数据集\\花菜类.csv')
df1.head()

In [58]:
predict = pd.read_excel('中间数据集\\未来七天成本预测结果.xlsx')
predict.columns = ['日期', '花菜类', '花叶类', '茄类', '辣椒类', '食用菌', '水生根茎类']
predict

,日期,花菜类,花叶类,茄类,辣椒类,食用菌,水生根茎类
0,未来第1天,7.916015,3.112513,4.920677,4.674565,7.618482,12.610124
1,未来第2天,7.928053,3.138526,4.926917,4.680828,7.565641,12.516030
2,未来第3天,7.924802,3.159227,4.909599,4.707002,7.651609,12.724465
3,未来第4天,7.912293,3.103637,4.939615,4.655270,7.595003,12.551193
4,未来第5天,7.916015,3.112513,4.920677,4.674565,7.618482,12.610124
5,未来第6天,7.928053,3.138526,4.926917,4.680828,7.565641,12.516030
6,未来第7天,7.924802,3.159227,4.909599,4.707002,7.651609,12.724465


In [59]:
range = pd.read_excel('中间数据集\\2_4_各品类对销量与成本加成定价的约束.xlsx')
range.head()

,品类,销量(千克)_下界,销量(千克)_上界,成本加成定价_下界,成本加成定价_上界
0,花菜类,2.493046,115.753382,3.718730,22.908976
1,花叶类,13.864352,471.022934,2.132901,10.954627
2,茄类,-0.344874,57.698732,4.173864,17.451418
3,辣椒类,3.853574,257.681711,2.470155,28.757278
4,食用菌,1.602830,250.875884,6.421218,18.767597


In [60]:
relate = pd.read_excel('中间数据集\\销量-成本加成定价线性关系.xlsx')
relate.columns = ['系数', '花菜类', '花叶类', '茄类', '辣椒类', '食用菌', '水生根茎类']
relate.head()

,系数,花菜类,花叶类,茄类,辣椒类,食用菌,水生根茎类
0,a,-0.039077,-0.003963,-0.065555,-0.032450,-0.032116,-0.046523
1,b,11.773336,7.148345,10.689466,12.245995,15.347841,12.402717


In [61]:
df1_predict = pd.DataFrame({
    '日期': predict['日期'],
    '预测批发价': predict['花菜类'],
    '损耗率_品类': df1['损耗率_品类'],
    }).dropna()
df1_predict['p/(1-k)'] = df1_predict['预测批发价'] / (1 - (df1_predict['损耗率_品类'] / 100))
df1_predict = df1_predict.round(2)
df1_predict

,日期,预测批发价,损耗率_品类,p/(1-k)
0,未来第1天,7.92,15.51,9.37
1,未来第2天,7.93,15.51,9.38
2,未来第3天,7.92,15.51,9.38
3,未来第4天,7.91,15.51,9.36
4,未来第5天,7.92,15.51,9.37
5,未来第6天,7.93,15.51,9.38
6,未来第7天,7.92,15.51,9.38


目标函数：求最大值

$$f(x, y) = x * (y - \dfrac{p}{1-k})$$

约束条件：

$$y = ax + b$$
$$lower\_i\_1 \leq x \leq upper\_i\_1$$
$$lower\_i\_2 \leq y \leq upper\_i\_2$$
$$x>0, y>0$$

<br><br>

## ***花菜类寻优***

<br><br>

**第一天**

In [ ]:
# 目标函数
def objective(x):
    return -(x * (relate['花菜类'][0] * x + relate['花菜类'][1] - df1_predict['p/(1-k)'][0]))

# 定义约束条件
cons = ({'type': 'ineq', 'fun': lambda x: x - range['销量(千克)_下界'][0]},
        {'type': 'ineq', 'fun': lambda x: range['销量(千克)_上界'][0] - x},
        {'type': 'ineq', 'fun': lambda x: relate['花菜类'][0] * x + relate['花菜类'][1] - range['成本加成定价_下界'][0]},  
        {'type': 'ineq', 'fun': lambda x: range['成本加成定价_上界'][0] - (relate['花菜类'][0] * x + relate['花菜类'][1])}
        )

# 优化
result = minimize(objective, x0=10, constraints=cons)

# 输出结果
optimal_x_1 = result.x[0]
optimal_y_1 = -objective(result.x[0])
max_f_x_1 = -result.fun

**第二天**

In [ ]:
# 目标函数
def objective(x):
    return -(x * (relate['花菜类'][0] * x + relate['花菜类'][1] - df1_predict['p/(1-k)'][1]))

# 定义约束条件
cons = ({'type': 'ineq', 'fun': lambda x: x - range['销量(千克)_下界'][0]},
        {'type': 'ineq', 'fun': lambda x: range['销量(千克)_上界'][0] - x},
        {'type': 'ineq', 'fun': lambda x: relate['花菜类'][0] * x + relate['花菜类'][1] - range['成本加成定价_下界'][0]},  
        {'type': 'ineq', 'fun': lambda x: range['成本加成定价_上界'][0] - (relate['花菜类'][0] * x + relate['花菜类'][1])}
        )

# 优化
result = minimize(objective, x0=10, constraints=cons)

# 输出结果
optimal_x_2 = result.x[0]
optimal_y_2 = -objective(result.x[0])
max_f_x_2 = -result.fun

**第三天**

In [ ]:
# 目标函数
def objective(x):
    return -(x * (relate['花菜类'][0] * x + relate['花菜类'][1] - df1_predict['p/(1-k)'][2]))

# 定义约束条件
cons = ({'type': 'ineq', 'fun': lambda x: x - range['销量(千克)_下界'][0]},
        {'type': 'ineq', 'fun': lambda x: range['销量(千克)_上界'][0] - x},
        {'type': 'ineq', 'fun': lambda x: relate['花菜类'][0] * x + relate['花菜类'][1] - range['成本加成定价_下界'][0]},  
        {'type': 'ineq', 'fun': lambda x: range['成本加成定价_上界'][0] - (relate['花菜类'][0] * x + relate['花菜类'][1])}
        )

# 优化
result = minimize(objective, x0=10, constraints=cons)

# 输出结果
optimal_x_3 = result.x[0]
optimal_y_3 = -objective(result.x[0])
max_f_x_3 = -result.fun

**第四天**


In [ ]:
# 目标函数
def objective(x):
    return -(x * (relate['花菜类'][0] * x + relate['花菜类'][1] - df1_predict['p/(1-k)'][3]))

# 定义约束条件
cons = ({'type': 'ineq', 'fun': lambda x: x - range['销量(千克)_下界'][0]},
        {'type': 'ineq', 'fun': lambda x: range['销量(千克)_上界'][0] - x},
        {'type': 'ineq', 'fun': lambda x: relate['花菜类'][0] * x + relate['花菜类'][1] - range['成本加成定价_下界'][0]},  
        {'type': 'ineq', 'fun': lambda x: range['成本加成定价_上界'][0] - (relate['花菜类'][0] * x + relate['花菜类'][1])}
        )

# 优化
result = minimize(objective, x0=10, constraints=cons)

# 输出结果
optimal_x_4 = result.x[0]
optimal_y_4 = -objective(result.x[0])
max_f_x_4 = -result.fun

**第五天**


In [ ]:
# 目标函数
def objective(x):
    return -(x * (relate['花菜类'][0] * x + relate['花菜类'][1] - df1_predict['p/(1-k)'][4]))

# 定义约束条件
cons = ({'type': 'ineq', 'fun': lambda x: x - range['销量(千克)_下界'][0]},
        {'type': 'ineq', 'fun': lambda x: range['销量(千克)_上界'][0] - x},
        {'type': 'ineq', 'fun': lambda x: relate['花菜类'][0] * x + relate['花菜类'][1] - range['成本加成定价_下界'][0]},  
        {'type': 'ineq', 'fun': lambda x: range['成本加成定价_上界'][0] - (relate['花菜类'][0] * x + relate['花菜类'][1])}
        )

# 优化
result = minimize(objective, x0=10, constraints=cons)

# 输出结果
optimal_x_5 = result.x[0]
optimal_y_5 = -objective(result.x[0])
max_f_x_5 = -result.fun

**第六天**


In [ ]:
# 目标函数
def objective(x):
    return -(x * (relate['花菜类'][0] * x + relate['花菜类'][1] - df1_predict['p/(1-k)'][5]))

# 定义约束条件
cons = ({'type': 'ineq', 'fun': lambda x: x - range['销量(千克)_下界'][0]},
        {'type': 'ineq', 'fun': lambda x: range['销量(千克)_上界'][0] - x},
        {'type': 'ineq', 'fun': lambda x: relate['花菜类'][0] * x + relate['花菜类'][1] - range['成本加成定价_下界'][0]},  
        {'type': 'ineq', 'fun': lambda x: range['成本加成定价_上界'][0] - (relate['花菜类'][0] * x + relate['花菜类'][1])}
        )

# 优化
result = minimize(objective, x0=10, constraints=cons)

# 输出结果
optimal_x_6 = result.x[0]
optimal_y_6 = -objective(result.x[0])
max_f_x_6 = -result.fun

**第七天**

In [ ]:
# 目标函数
def objective(x):
    return -(x * (relate['花菜类'][0] * x + relate['花菜类'][1] - df1_predict['p/(1-k)'][6]))

# 定义约束条件
cons = ({'type': 'ineq', 'fun': lambda x: x - range['销量(千克)_下界'][0]},
        {'type': 'ineq', 'fun': lambda x: range['销量(千克)_上界'][0] - x},
        {'type': 'ineq', 'fun': lambda x: relate['花菜类'][0] * x + relate['花菜类'][1] - range['成本加成定价_下界'][0]},  
        {'type': 'ineq', 'fun': lambda x: range['成本加成定价_上界'][0] - (relate['花菜类'][0] * x + relate['花菜类'][1])}
        )

# 优化
result = minimize(objective, x0=10, constraints=cons)

# 输出结果
optimal_x_7 = result.x[0]
optimal_y_7 = -objective(result.x[0])
max_f_x_7 = -result.fun

<br><br><br>

In [ ]:
result_1 = pd.DataFrame({
    '最优销量': [optimal_x_1, optimal_x_2, optimal_x_3, optimal_x_4, optimal_x_5, optimal_x_6, optimal_x_7],
    '最优定价': [optimal_y_1, optimal_y_2, optimal_y_3, optimal_y_4, optimal_y_5, optimal_y_6, optimal_y_7], 
    '最大收益': [max_f_x_1, max_f_x_2, max_f_x_3, max_f_x_4, max_f_x_5, max_f_x_6, max_f_x_7],
    }, index=['第一天', '第二天', '第三天', '第四天', '第五天', '第六天', '第七天'])
result_1['最优补货量'] = result_1['最优销量'] / (1 - df1_predict['损耗率_品类'][0] / 100)

# 写入结果
result_1.to_excel("结果\\2_寻优结果_花菜类.xlsx")
result_1

,最优销量,最优定价,最大收益,最优补货量
第一天,30.751688,36.953286,36.953286,36.396838
第二天,30.623682,36.646409,36.646409,36.245334
第三天,30.623682,36.646409,36.646409,36.245334
第四天,30.879589,37.261442,37.261442,36.548218
第五天,30.751688,36.953286,36.953286,36.396838
第六天,30.623682,36.646409,36.646409,36.245334
第七天,30.623682,36.646409,36.646409,36.245334


# 其他五种蔬菜均如此，我懒得加了